In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, timedelta

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/Transactions.csv')
df1.sample(5)

,Transaction_ID,Type,Transaction_Type,Customer_Name,Total,Success,Day,Transaction_Notes,Source,Country,Auth_code
137,5432109,Charge,Friends & Family,Mia,2882,1,6/1/2023,Thank you,Desktop,US,O9V1C4DJ
107,2345678,Refund,Goods and Services,Alexander,1740,1,5/7/2023,Thank you,Desktop,US,V7Y9R5KM
186,3210987,Charge,Goods and Services,Olivia,4567,1,7/22/2023,Thank you,Desktop,US,W8Y9N0VJ
33,5678901,Refund,Friends & Family,Lily,2347,1,2/5/2023,Thanks,Tablet,US,G9S5P2XK
134,2109876,Charge,Friends & Family,Jacob,3946,1,5/29/2023,Thanks,Desktop,US,G0J7Q8HP


# **Data Precleaning**


In [3]:
df1.drop(['Transaction_ID', 'Auth_code'], axis=1, inplace= True)

In [4]:
df1 =df1[df1['Success'] == 1] # we are not interested in the transcation that failed

In [5]:
df1['Transaction_Notes'].fillna("N/A", inplace= True)

<ipython-input-5-81aff8d5b68c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Transaction_Notes'].fillna("N/A", inplace= True)


In [6]:
df1['Day'] = pd.to_datetime(df1['Day'])

<ipython-input-6-d9c769582a1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Day'] = pd.to_datetime(df1['Day'])


In [7]:
df1.columns

Index(['Type', 'Transaction_Type', 'Customer_Name', 'Total', 'Success', 'Day',
       'Transaction_Notes', 'Source', 'Country'],
      dtype='object')

In [8]:
#rearranging the columns
df = df1.loc[:,['Total','Transaction_Type','Type','Country','Source', 'Day','Customer_Name','Transaction_Notes' ]]

In [9]:
df.head()

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Transaction_Notes
0,3286,Goods and Services,Charge,US,Tablet,2023-01-02,James,Thanks
1,1624,Friends & Family,Refund,US,Phone,2023-01-03,Emily,Raffle
2,2659,Goods and Services,Charge,US,Desktop,2023-01-04,Liam,Thanks
3,4897,Goods and Services,Charge,US,Phone,2023-01-05,Olivia,Thanks
4,3643,Friends & Family,Charge,UK,Desktop,2023-01-06,Benjamin,Thanks


# **Data Manipulations**
  ## **Transactions Analysis**

In [10]:
totalsum = np.sum(df['Total'])

In [71]:
total_transactions = df['Type'].count()

In [66]:
mean_transaction =np.mean(df['Total'])
median_transaction =np.median(df['Total'])
max_transaction =np.max(df['Total'])

In [67]:
chargeonlytransaction = df[df['Type'] == 'Charge']
refundonlytransaction = df[df['Type'] == 'Refund']
chargebackonlytransaction = df[df['Type'] == 'Chargeback']

In [13]:
#Getting the dates 90 and 180 days prior to the current date for analysis

days90=pd.to_datetime(date.today()-timedelta(days=90))
days180 =pd.to_datetime(date.today()-timedelta(days=180))

In [68]:
chargetotal = np.sum(chargeonlytransaction['Total'])
refundtotal = np.sum(refundonlytransaction['Total'])
chargebacktotal = np.sum(chargebackonlytransaction['Total'])

In [69]:
# Retrieving the total transactions for charges, refunds, and chargebacks occurring in the last 90 and 180 days

charge90days = np.sum(chargeonlytransaction[chargeonlytransaction['Day'] > days90]['Total'])
charge180days = np.sum(chargeonlytransaction[chargeonlytransaction['Day'] > days180]['Total'])

refund90days = np.sum(refundonlytransaction[refundonlytransaction['Day'] > days90]['Total'])
refund180days = np.sum(refundonlytransaction[refundonlytransaction['Day'] > days180]['Total'])

chargeback90days = np.sum(chargebackonlytransaction[chargebackonlytransaction['Day'] > days90]['Total'])
chargeback180days = np.sum(chargebackonlytransaction[chargebackonlytransaction['Day'] > days180]['Total'])

In [16]:
refundratelifetime = (refundtotal/chargetotal)
refundrate90days = (refund90days/charge90days)
refundrate180days = (refund180days/charge180days)

chargebackratelifetime = (chargebacktotal/chargetotal)
chargebackrate90days = (chargeback90days/charge90days)
chargebackrate180days = (chargeback180days/charge180days)


<ipython-input-16-271829e3029b>:2: RuntimeWarning: invalid value encountered in scalar divide
  refundrate90days = (refund90days/charge90days)
<ipython-input-16-271829e3029b>:3: RuntimeWarning: invalid value encountered in scalar divide
  refundrate180days = (refund180days/charge180days)
<ipython-input-16-271829e3029b>:6: RuntimeWarning: invalid value encountered in scalar divide
  chargebackrate90days = (chargeback90days/charge90days)
<ipython-input-16-271829e3029b>:7: RuntimeWarning: invalid value encountered in scalar divide
  chargebackrate180days = (chargeback180days/charge180days)


# **Pivot Tables**

In [21]:
# Creating a pivot table to summarize total transaction amounts and counts per customer

pivottablenames =pd.pivot_table(df, index = ['Customer_Name'], aggfunc ={'Total':np.sum, 'Customer_Name':'count', })
pivottablenames =pivottablenames.rename(columns ={"Customer_Name":"count_of_total","Total":"sum_of_total"})
pivottablenames = pivottablenames.loc[:, ['sum_of_total','count_of_total']]
pivottablenames

,sum_of_total,count_of_total
Customer_Name,,
Abigail,18119,6
Alexander,26505,7
Amelia,1757,1
Andrew,31066,8
Ava,7367,2
Benjamin,22737,7
Charlotte,21762,6
Chloe,15873,5
Christopher,19553,7


In [25]:
total_unique_customers = pivottablenames['count_of_total'].count()
total_unique_customers

37

In [26]:
avg_transactions_count_per_customer = np.mean(pivottablenames['count_of_total'])
avg_transactions_sum_per_customer = np.mean(pivottablenames['sum_of_total'])


In [36]:
# Generating a pivot table to analyze transaction types along with their respective counts and total amounts

pivottabletransactiontype =pd.pivot_table(df, index =['Transaction_Type'] , aggfunc ={'Transaction_Type':'count','Total':np.sum })
pivottabletransactiontype.columns =['Transaction_Type', 'Total']
pivottabletransactiontype

,Transaction_Type,Total
Transaction_Type,,
Friends & Family,238208,74
Goods and Services,373295,121


In [43]:
#pivottabletransactiontype['total_percent']= (pivottabletransactiontype['Total']/totalsum).apply('{:.2%}'.format)
pivottabletransactiontype['total_percent']= (pivottabletransactiontype['Total']/totalsum).apply(lambda x:f"{x:.2%}")
pivottabletransactiontype

,Transaction_Type,Total,total_percent
Transaction_Type,,,
Friends & Family,238208,74,0.01%
Goods and Services,373295,121,0.02%


In [45]:
#Creating a pivot table to analyze transaction counts and total amounts by country, along with the percentage of total sum each country contributes

pivottabltransactioncountry = pd.pivot_table(df, index=['Country'], aggfunc={'Country': 'count', 'Total': np.sum})
pivottabltransactioncountry['totalpercent'] = (pivottabltransactioncountry['Total']/totalsum).apply(lambda x: f"{x:.2%}")
pivottabltransactioncountry

,Country,Total,totalpercent
Country,,,
AU,1,3792,0.62%
UK,8,27967,4.57%
US,186,579744,94.81%


In [49]:
firstname = 'JOHN'
namefinal =df[df['Customer_Name'].str.contains(firstname, case =False)]

In [50]:
namefinal

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Transaction_Notes
32,3819,Goods and Services,Charge,US,Desktop,2023-02-04,John,Thanks


In [57]:
# Filter transactions with notes containing flagged words

payment_note =df[df['Transaction_Notes'].isna() == False]

flagged_words = 'raffle|razz|lottery'
payment_note_final =payment_note[payment_note['Transaction_Notes'].str.contains(flagged_words , case = False)]

In [55]:
payment_note_final

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Transaction_Notes
1,1624,Friends & Family,Refund,US,Phone,2023-01-03,Emily,Raffle
5,1320,Goods and Services,Charge,US,Desktop,2023-01-07,Ryan,Razz
8,450,Friends & Family,Charge,US,Phone,2023-01-13,William,Lottery
15,1757,Goods and Services,Charge,US,Desktop,2023-01-21,Amelia,Lottery
23,3191,Goods and Services,Charge,US,Phone,2023-01-27,Ella,Lottery
24,3563,Goods and Services,Charge,US,Desktop,2023-01-29,Christopher,Lottery
25,3962,Friends & Family,Refund,US,Desktop,2023-01-30,Grace,Lottery
26,4565,Goods and Services,Charge,US,Phone,2023-01-31,Matthew,Lottery
41,4966,Friends & Family,Refund,US,Phone,2023-02-16,Emily,Razz
49,1967,Friends & Family,Charge,US,Desktop,2023-02-24,Harper,Razz


In [58]:
# Analyzing high-value transactions

highticketval =3500

highticket = df[df['Total'] >= highticketval].copy()
highticket =highticket.sort_values(by ='Total', ascending =False)
highticket

,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Transaction_Notes
101,4996,Goods and Services,Charge,US,Desktop,2023-05-01,Benjamin,Thank you
122,4993,Goods and Services,Charge,US,Tablet,2023-05-17,Chloe,Thanks
144,4990,Goods and Services,Charge,US,Desktop,2023-06-08,Elizabeth,Thank you
116,4971,Friends & Family,Charge,US,Desktop,2023-05-16,Daniel,N/A
41,4966,Friends & Family,Refund,US,Phone,2023-02-16,Emily,Razz
...,...,...,...,...,...,...,...,...
59,3661,Goods and Services,Charge,US,Desktop,2023-03-07,Victoria,Thank you
4,3643,Friends & Family,Charge,UK,Desktop,2023-01-06,Benjamin,Thanks
24,3563,Goods and Services,Charge,US,Desktop,2023-01-29,Christopher,Lottery
71,3543,Goods and Services,Charge,US,Tablet,2023-03-22,Benjamin,Thanks


In [63]:
# Identifying potential instances of consecutive high-value transactions or suspicious customer behavior

dup =df.copy()

dup['Customer_Name_next'] = dup['Customer_Name'].shift(1)
dup['Customer_Name_prev'] = dup['Customer_Name'].shift(-1)

dup['created_at_day'] = dup['Day']
dup['created_at_dayprev'] = dup['Day'].shift(-1)
dup['created_at_daynext'] = dup['Day'].shift(1)

dup3 = dup.query('(created_at_day == created_at_dayprev | created_at_day == created_at_daynext) & (Customer_Name == Customer_Name_next | Customer_Name == Customer_Name_prev)')
dup3


,Total,Transaction_Type,Type,Country,Source,Day,Customer_Name,Transaction_Notes,Customer_Name_next,Customer_Name_prev,created_at_day,created_at_dayprev,created_at_daynext
30,4285,Goods and Services,Charge,US,Desktop,2023-02-04,Andrew,Thanks,Victoria,Andrew,2023-02-04,2023-02-04,2023-02-03
31,2193,Goods and Services,Charge,US,Phone,2023-02-04,Andrew,Thanks,Andrew,John,2023-02-04,2023-02-04,2023-02-04
60,2793,Friends & Family,Refund,US,Desktop,2023-03-07,Christopher,Thank you,Victoria,Christopher,2023-03-07,2023-03-07,2023-03-07
61,2562,Friends & Family,Charge,US,Phone,2023-03-07,Christopher,Thank you,Christopher,David,2023-03-07,2023-03-09,2023-03-07
125,4241,Goods and Services,Charge,US,Tablet,2023-05-20,Andrew,Thanks,Sofia,Andrew,2023-05-20,2023-05-20,2023-05-19
126,4241,Friends & Family,Charge,US,Phone,2023-05-20,Andrew,Thanks,Andrew,William,2023-05-20,2023-05-22,2023-05-20


In [72]:
dfcalc = pd.DataFrame({'totalsum':[totalsum],
                           'mean_transaction':[mean_transaction],
                           'median_transaction':[median_transaction],
                           'max_transaction':[max_transaction],
                           'total_transactions':[total_transactions],
                           'chargetotal':[chargetotal],
                           'charge90days':[charge90days],
                           'charge180days':[charge180days],
                           'refundtotal':[refundtotal],
                           'refund90days':[refund90days],
                           'refund180days':[refund180days],
                           'refundrateliefetime':[refundratelifetime],
                           'refundrate90days':[refundrate90days],
                           'refundrate180days':[refundrate180days],
                           'chargebacktotal':[chargebacktotal],
                           'chargeback90days':[chargeback90days],
                           'chargeback180days':[chargeback180days],
                           'chargebackrateliefetime':[chargebackratelifetime],
                           'chargebackrate90days':[chargebackrate90days],
                           'chargebackrate180days':[chargebackrate180days],
                           'total_unique_customer_names':[total_unique_customers],
                           'avg_transactions_count_per_customer_name':[avg_transactions_count_per_customer],
                           'avg_transactions_sum_per_customer_name':[avg_transactions_sum_per_customer],
                           '90 Days':[days90],
                           '180 Days':[days180],
                        })

In [74]:
format_mapping = {"totalsum": '${:,.2f}',
                  "mean_transaction": '${:,.2f}',
                  "median_transaction": '${:,.2f}',
                  "max_transaction": '${:,.2f}',
                  "total_transactions": '{:,.0f}',
                  'chargetotal': '${:,.2f}',
                  'charge90days': '${:,.2f}',
                  'charge180days': '${:,.2f}',
                  'refundtotal': '${:,.2f}',
                  'refund90days': '${:,.2f}',
                  'refund180days': '${:,.2f}',
                  'refundrateliefetime':'{:.2%}',
                  'refundrate90days':'{:.2%}',
                  'refundrate180days':'{:.2%}',
                  'chargebacktotal':'${:,.2f}',
                  'chargeback90days':'${:,.2f}',
                  'chargeback180days':'${:,.2f}',
                  'chargebackrateliefetime':'{:.2%}',
                  'chargebackrate90days':'{:.2%}',
                  'chargebackrate180days':'{:.2%}',
                  "total_unique_customer_names": '{:,.0f}',
                  "avg_transactions_count_per_customer_name": '{:,.2f}',
                  "avg_transactions_sum_per_customer_name": '${:,.2f}',
                    }